In [9]:
import os
import librosa
import pandas as pd
import numpy as np
from tqdm import tqdm 

In [12]:
df = pd.read_csv('./full_test_split.csv')
print(df.head())

   Participant_ID  PHQ8_Binary  PHQ_Score  Gender
0             300            0          2       1
1             301            0          3       1
2             306            0          0       0
3             308            1         22       0
4             309            1         15       1


In [13]:
# Definirea funcției pentru a crea severitatea
def determine_severity(score):
    if 0 <= score <= 4:
        return 0
    elif 5 <= score <= 9:
        return 1
    elif 10 <= score <= 14:
        return 2
    elif 15 <= score <= 19:
        return 3
    elif 20 <= score <= 24:
        return 4
    else:
        return None

In [14]:
# Aplicarea funcției pe coloana PHQ_Score pentru a crea coloana Severity
df['Severity'] = df['PHQ_Score'].apply(determine_severity)
print(df.head())

   Participant_ID  PHQ8_Binary  PHQ_Score  Gender  Severity
0             300            0          2       1         0
1             301            0          3       1         0
2             306            0          0       0         0
3             308            1         22       0         4
4             309            1         15       1         3


In [15]:
print(df.value_counts('Severity'))

Severity
0    22
1    11
3     7
2     5
4     2
Name: count, dtype: int64


In [16]:
# Function to process each WAV file
def process_wav(file_path, sr=16000, frame_duration=7):
    # Load the WAV file
    y, sr = librosa.load(file_path, sr=sr)
    
    # Eliminate the first 15 seconds
    y = y[int(sr*15):]
    
    # Calculate the number of frames
    frame_size = sr * frame_duration
    num_frames = len(y) // frame_size
    
    # Segment the WAV file into frames
    frames = [y[i*frame_size:(i+1)*frame_size] for i in range(num_frames)]
    
    return frames

# Function to get labels for WAV file frames based on participant ID
def get_labels_for_frames(participant_id, data_df, num_frames):
    labels = []
    participant_row = data_df[data_df['Participant_ID'] == int(participant_id)]
    if not participant_row.empty:
        severity = participant_row.iloc[0]['Severity']
        labels = [severity] * num_frames
    
    return labels


In [17]:

wav_folder_path = 'E:\\Daicwoz\\after_vad_speech\\'

data_df = df

data = []
labels = []


participant_ids = set(data_df['Participant_ID'].astype(str))

for file_name in tqdm(os.listdir(wav_folder_path)):
    if file_name.endswith('.wav'):

        participant_id = file_name.split('_')[0]
        

        if participant_id in participant_ids:
            file_path = os.path.join(wav_folder_path, file_name)
            
            frames = process_wav(file_path)
            
            frame_labels = get_labels_for_frames(participant_id, data_df, len(frames))
            
            data.append(frames)
            labels.append(frame_labels)


100%|██████████| 187/187 [00:01<00:00, 127.95it/s]


In [20]:
print(len(data), len(labels))
# print(data[0].shape)
data_vector = np.concatenate(data, axis=0)
print(data_vector.shape)


47 47
(4033, 112000)


In [21]:
print(labels[0])
labels = np.concatenate(labels, axis=0)
print(labels.shape)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
(4033,)


In [24]:
np.save('data_severity.npy', data_vector)
np.save('labels_severity.npy', labels)

In [25]:
data = np.load('data_severity.npy', allow_pickle=True)
labels = np.load('labels_severity.npy', allow_pickle=True)

print(data.shape)
print(labels.shape)

(4033, 112000)
(4033,)
